In [ ]:
from kafka import KafkaConsumer
import pandas as pd

# To consume latest messages and auto-commit offsets
consumer2 = KafkaConsumer('weather_from_API', bootstrap_servers=['localhost:9093'], auto_offset_reset='latest')
from datetime import datetime
import time
# subscribe to the topic and save the data to a csv file
curr_time = datetime.now()
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'])
session = cluster.connect()
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS airpollution 
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")
session.set_keyspace('airpollution')
session.execute("""
        CREATE TABLE IF NOT EXISTS weather_data (
            district text,
            timestamp bigint,
            temp float,
            feels_like float,
            pressure int,
            humidity int,
            clouds int,
            wind_speed float,
            wind_deg int,
            weather_main text,
            weather_desc text,
            PRIMARY KEY (district, timestamp)
        )
    """)


while True:
    for message in consumer2:
        print(f"Message received: {message.value}")
        data = {}
        import json
        message_value = json.loads(message.value.decode('utf-8'))
        data['district'] = message_value['district']
        data['timestamp'] = message_value['timestamp']
        data['temp'] = message_value['temp']
        data['feels_like'] = message_value['feels_like']
        data['pressure'] = message_value['pressure']
        data['humidity'] = message_value['humidity']
        data['clouds'] = message_value['clouds']
        data['wind_speed'] = message_value['wind_speed']
        data['wind_deg'] = message_value['wind_deg']
        data['weather_main'] = message_value['weather_main']
        data['weather_desc'] = message_value['weather_desc']
        print(f"Data to insert: {data}")
        df = pd.DataFrame(data, index=[0])
        df.to_csv('weather_from_API.csv', mode='a', header=False)
        print("Inserting data into Cassandra...")
        session.execute("""
            INSERT INTO weather_data (district, timestamp, temp, feels_like, pressure, humidity, clouds, wind_speed, wind_deg, weather_main, weather_desc)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (data['district'], data['timestamp'], data['temp'], data['feels_like'], data['pressure'], data['humidity'], data['clouds'], data['wind_speed'], data['wind_deg'], data['weather_main'], data['weather_desc']))
        
        
        